## Import Libraries

In [1]:
import pandas as pd
import os
from pdfminer.high_level import extract_text
import sqlite3

## Load Metadata

In [2]:
metadata = pd.read_csv('../../wetransfer_b_2022-12-09_1647/B/CLA_meta_from_2018.csv')

## Build Dictionary of PDF Files

In [3]:
file_dict = {}

"""With filename as key, store path to PDF files in a dictionary"""

for subdir, dirs, files in os.walk('../../wetransfer_a_2022-12-09_1528/A'):
    for file in files:
        path = subdir + '/' + file
        file_dict[file] = path

for subdir, dirs, files in os.walk('../../wetransfer_b_2022-12-09_1647/B'):
    for file in files:
        path = subdir + '/' + file
        file_dict[file] = path

## Map Dictionary to Metadata

In [ ]:
files = metadata[['jc_number', 'filename']]
files['path'] = files['filename'].map(file_dict) ### map path to PDF files to metadata via filename
files['path'] = files['path'].str.replace('\\', '/')
jc_number_list = files['jc_number'].unique()
files_group = files.groupby(files['jc_number'])   ### group pdf files by jc_number

## Build and/or Connect to Database

In [10]:
database = '../../database'
connect = sqlite3.connect(database)
metadata.to_sql(name='metadata', con=connect, if_exists='replace')

## Extract Text from PDF Files and Store into CSV and Database

In [ ]:
for number in jc_number_list:   ### loop through each jc_number
    grouped_dict = {}
    grouped_data = files_group.get_group(number)
    for p in grouped_data['path']:   ### loop through each path per jc_number
        text = extract_text(p)   ### extract pdf text using pdfminer.six
        grouped_dict[p] = text   ### with path as key, store pdf text in a dictionary
    grouped_data['pdf_text'] = grouped_data['path'].map(grouped_dict) ### map pdf text to dataframe
    grouped_data.to_sql(name='files', con=connect, if_exists='append')   ### store data in database
    string_number = str(number)
    grouped_data.to_csv(f'../../data/jc_{string_number[0:3]}_{string_number[4:]}.csv')   ### save data as csv file